# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [195]:
import pandas as pd
import plotly.express as px

# Notebook Presentation

In [196]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [197]:
df_apps = pd.read_csv('apps.csv')

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [198]:
print(df_apps.shape)
print(df_apps.columns)

(10841, 12)
Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Android_Ver'],
      dtype='object')


In [199]:
df_apps.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
4268,Stickman and Axe,GAME,4.40,69,2.80,"50,000",Free,0,Teen,Action,"November 25, 2017",4.0.3 and up
1813,Diabetes & Diet Tracker,MEDICAL,4.60,395,19.00,"1,000",Paid,$9.99,Everyone,Medical,"July 16, 2018",5.0 and up
2912,myNaida CI,MEDICAL,4.60,64,57.00,"5,000",Free,0,Everyone,Medical,"July 10, 2018",5.0 and up
192,BU School of Public Health,TRAVEL_AND_LOCAL,NaN,0,29.00,10,Free,0,Everyone,Travel & Local,"May 20, 2018",4.4 and up
2533,CT Scan Generations,FAMILY,4.70,6,5.10,"1,000",Free,0,Everyone,Education,"April 1, 2018",4.0 and up


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns.

In [200]:
df_apps = df_apps.drop(["Last_Updated"], axis=1)
df_apps = df_apps.drop(["Android_Ver"], axis=1)

In [201]:
#check if dropped correctly
print(df_apps.shape)
print(df_apps.columns)

(10841, 10)
Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres'],
      dtype='object')


### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows.

In [202]:
print(df_apps.isna().sum())
# print(df_apps['Rating'].isna().sum())


App                  0
Category             0
Rating            1474
Reviews              0
Size_MBs             0
Installs             0
Type                 1
Price                0
Content_Rating       0
Genres               0
dtype: int64


In [203]:
df_apps_clean = df_apps.dropna()
print(df_apps_clean.isna().any())
df_apps_clean.shape

App               False
Category          False
Rating            False
Reviews           False
Size_MBs          False
Installs          False
Type              False
Price             False
Content_Rating    False
Genres            False
dtype: bool


(9367, 10)

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`.


In [204]:
duplicated_rows = df_apps_clean[df_apps_clean.duplicated()]

In [205]:
duplicated_rows.shape

(476, 10)

In [206]:
duplicated_rows.head()


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
946,420 BZ Budeze Delivery,MEDICAL,5.00,2,11.00,100,Free,0,Mature 17+,Medical
1133,MouseMingle,DATING,2.70,3,3.90,100,Free,0,Mature 17+,Dating
1196,"Cardiac diagnosis (heart rate, arrhythmia)",MEDICAL,4.40,8,6.50,100,Paid,$12.99,Everyone,Medical
1231,Sway Medical,MEDICAL,5.00,3,22.00,100,Free,0,Everyone,Medical
1247,Chat Kids - Chat Room For Kids,DATING,4.70,6,4.90,100,Free,0,Mature 17+,Dating


In [207]:
df_apps_clean = df_apps_clean.drop_duplicates(subset=['App','Type','Price'])

In [208]:
df_apps_clean.shape

(8199, 10)

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [209]:
df_apps_clean.sort_values("Rating", ascending=False).head(10)[['App','Rating', 'Reviews','Installs']]
#the problem is the can have low amount of installs/reviews (du'h)

,App,Rating,Reviews,Installs
21,KBA-EZ Health Guide,5.00,4,1
1230,Sway Medical,5.00,3,100
1227,AJ Men's Grooming,5.00,2,100
1224,FK Dedinje BGD,5.00,36,100
1223,CB VIDEO VISION,5.00,13,100
1222,"Beacon Baptist Jupiter, FL",5.00,14,100
1214,BV Mobile Apps,5.00,3,100
2680,Florida Wildflowers,5.00,5,"1,000"
1206,ADS-B Driver,5.00,2,100
2750,"Superheroes, Marvel, DC, Comics, TV, Movies News",5.00,34,"5,000"


# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please?

In [210]:
df_apps_clean.sort_values('Size_MBs', ascending=False).head(10)[['App','Size_MBs']]
#yup limit is 100

,App,Size_MBs
9942,Talking Babsy Baby: Baby Games,100.00
10687,Hungry Shark Evolution,100.00
9943,Miami crime simulator,100.00
9944,Gangster Town: Vice District,100.00
3144,Vi Trainer,100.00
9945,Ultimate Tennis,100.00
7926,Post Bank,100.00
7927,The Walking Dead: Our World,100.00
7928,Stickman Legends: Shadow Wars,100.00
4176,Car Crash III Beam DH Real Damage Simulator 2018,100.00


# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [211]:
df_apps_clean.sort_values('Reviews',ascending=False).head(5)[['App','Reviews','Type']]

,App,Reviews,Type
10805,Facebook,78158306,Free
10785,WhatsApp Messenger,69119316,Free
10806,Instagram,66577313,Free
10784,Messenger – Text and Video Chat for Free,56642847,Free
10650,Clash of Clans,44891723,Free


# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [212]:
ratings = df_apps_clean['Content_Rating'].value_counts()
ratings

,count
Content_Rating,
Everyone,6621
Teen,912
Mature 17+,357
Everyone 10+,305
Adults only 18+,3
Unrated,1


In [213]:
fig = px.pie(labels = ratings.index, values=ratings.values,title="Content Rating",names=ratings.index)
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [214]:
fig = px.pie(labels = ratings.index, values=ratings.values,title="Content Rating",names=ratings.index, hole=0.6)
fig.update_traces(textposition='inside', textfont_size=15, textinfo='percent')

fig.show()

# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install?

Check the datatype of the Installs column.

Count the number of apps at each level of installations.

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first.

In [215]:
df_apps_clean[['Installs','App']].groupby('Installs').count()

,App
Installs,
1,3
"1,000",698
"1,000,000",1417
"1,000,000,000",20
10,69
"10,000",988
"10,000,000",933
100,303
"100,000",1096


In [216]:
df_apps_clean['Installs'] = df_apps_clean.Installs.astype(str).str.replace(',', "")
df_apps_clean['Installs'] = pd.to_numeric(df_apps_clean['Installs'])

In [217]:
df_apps_clean[['Installs','App']].groupby('Installs').count()

,App
Installs,
1,3
5,9
10,69
50,56
100,303
500,199
1000,698
5000,425
10000,988


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [218]:
df_apps_clean['Price'] = df_apps_clean['Price'].astype(str).str.replace('$',"")
df_apps_clean['Price'] = pd.to_numeric(df_apps_clean['Price'])
df_apps_clean[['Price', 'App']]

df_apps_clean = df_apps_clean[df_apps_clean['Price'] < 250]

df_apps_clean['Revenue_Estimate'] = df_apps_clean.Installs.mul(df_apps_clean.Price)
df_apps_clean.sort_values('Revenue_Estimate', ascending=False)[:10][['App','Revenue_Estimate','Installs','Price']]


<ipython-input-218-f54c80278512>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,App,Revenue_Estimate,Installs,Price
9220,Minecraft,"69,900,000.00",10000000,6.99
8825,Hitman Sniper,"9,900,000.00",10000000,0.99
7151,Grand Theft Auto: San Andreas,"6,990,000.00",1000000,6.99
7477,Facetune - For Free,"5,990,000.00",1000000,5.99
7977,Sleep as Android Unlock,"5,990,000.00",1000000,5.99
6594,DraStic DS Emulator,"4,990,000.00",1000000,4.99
6082,Weather Live,"2,995,000.00",500000,5.99
7954,Bloons TD 5,"2,990,000.00",1000000,2.99
7633,Five Nights at Freddy's,"2,990,000.00",1000000,2.99
6746,Card Wars - Adventure Time,"2,990,000.00",1000000,2.99


# Plotly Bar Charts & Scatter Plots: Analysing App Categories

In [219]:
df_apps_clean["Category"].nunique()

33

In [220]:
top10_category = df_apps_clean["Category"].value_counts()[:10]
top10_category

,count
Category,
FAMILY,1606
GAME,910
TOOLS,719
PRODUCTIVITY,301
PERSONALIZATION,298
LIFESTYLE,297
FINANCE,296
MEDICAL,292
PHOTOGRAPHY,263


In [221]:
bar = px.bar(x=top10_category.index, y=top10_category.values)

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [222]:
bar.show()

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [223]:
category_installs = df_apps_clean.groupby('Category').agg({"Installs":pd.Series.sum})

In [224]:
category_installs.sort_values('Installs', ascending=True, inplace=True)

In [225]:
h_bar = px.bar(x = category_installs.Installs,
               y = category_installs.index,
               orientation='h',
               title='Category Popularity')

h_bar.update_layout(xaxis_title='Number of Downloads', yaxis_title='Category')
h_bar.show()

### Category Concentration - Downloads vs. Competition

In [226]:
category_number = df_apps_clean.groupby('Category').agg({"App":pd.Series.count})

In [227]:
merged_df = pd.merge(category_number, category_installs, on='Category', how='inner')

In [228]:
print(f'The dimensions of the DataFrame are: {merged_df.shape}')


The dimensions of the DataFrame are: (33, 2)


In [229]:
scatter = px.scatter(merged_df, x='App', y ="Installs", title='Category concentration',
                    hover_name=merged_df.index,
                     size='App', color='Installs')


In [230]:
scatter.update_layout(xaxis_title="Number of Apps (Lower=More Concentrated)",
                      yaxis_title="Installs",
                      yaxis=dict(type='log'))
scatter.show()

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html).


In [231]:
df_apps_clean['Genres'].value_counts()

,count
Genres,
Tools,718
Entertainment,467
Education,429
Productivity,301
Personalization,298
...,...
Adventure;Brain Games,1
Travel & Local;Action & Adventure,1
Art & Design;Pretend Play,1


In [232]:
stack = df_apps_clean.Genres.str.split(';', expand=True).stack()
num_genres = stack.value_counts()
len(num_genres)


53

# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data?

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/).

* Find a way to set the colour scale using the color_continuous_scale parameter.
* Find a way to make the color axis disappear by using coloraxis_showscale.

In [233]:
bar = px.bar(x = num_genres.index[:15], # index = category name
             y = num_genres.values[:15], # count
             title='Top Genres',
             hover_name=num_genres.index[:15],
             color=num_genres.values[:15],
             color_continuous_scale='Agsunset')

bar.update_layout(xaxis_title='Genre',
yaxis_title='Number of Apps',
coloraxis_showscale=True)

bar.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [234]:
df_apps_clean.Type.value_counts()


,count
Type,
Free,7595
Paid,589


In [235]:
df_free_vs_paid = df_apps_clean.groupby(["Category", "Type"], as_index=False).agg({'App':pd.Series.count})


In [236]:
df_free_vs_paid.head()

,Category,Type,App
0,ART_AND_DESIGN,Free,58
1,ART_AND_DESIGN,Paid,3
2,AUTO_AND_VEHICLES,Free,72
3,AUTO_AND_VEHICLES,Paid,1
4,BEAUTY,Free,42


In [237]:
g_bar = px.bar(df_free_vs_paid,
               x='Category',
               y='App',
               title='Free vs Paid Apps by Category',
               color='Type',
               barmode='group')

In [238]:
g_bar.update_layout(xaxis_title='Category',
                    yaxis_title='Number of Apps',
                    xaxis={'categoryorder':'total descending'},
                    yaxis=dict(type='log'))
g_bar.show()

# Plotly Box Plots: Lost Downloads for Paid Apps

In [239]:
box = px.box(df_apps_clean,
             y='Installs',
             x='Type',
             color='Type',
             notched=True,
             points='all',
             title='How Many Downloads are Paid Apps Giving Up?')

In [240]:
box.update_layout(yaxis=dict(type='log'))

box.show()

# Plotly Box Plots: Revenue by App Category

In [241]:
df_paid_apps = df_apps_clean[df_apps_clean['Type'] == 'Paid']
box = px.box(df_paid_apps,
             x='Category',
             y='Revenue_Estimate',
             title='How Much Can Paid Apps Earn?')



In [242]:
box.update_layout(xaxis_title='Category',
                  yaxis_title='Paid App Ballpark Revenue',
                  xaxis={'categoryorder':'min ascending'},
                  yaxis=dict(type='log'))

box.show()

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.

In [243]:
box = px.box(df_paid_apps, x='Category', y='Price', title = 'Price per Category')

In [244]:
box.update_layout(xaxis_title='Category',
                  yaxis_title='Paid App Price',
                  xaxis={'categoryorder':'max descending'},
                  yaxis=dict(type='log'))
box.show()